In [ ]:
%load_ext autoreload
%autoreload
from retro_branching.environments import EcoleBranching
from retro_branching.agents import StrongBranchingAgent, PseudocostBranchingAgent, RandomAgent
from retro_branching.utils import seed_stochastic_modules_globally, PlotAesthetics

import ecole

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from tabulate import tabulate

import glob
from collections import defaultdict
import time
import pickle

In [ ]:
# set params

agent = RandomAgent()
# agent = PseudocostBranchingAgent()

co_class = 'set_covering'
co_class_kwargs = {'n_rows': 500, 'n_cols': 1000}

verbose = True

In [ ]:
# load instances

instances_path = f'/scratch/datasets/retro_branching/instances/{co_class}'
for key,val in co_class_kwargs.items():
    instances_path += f'_{key}_{val}'
files = glob.glob(instances_path + '/*.mps')
print(f'Loaded {len(files)} instance files from {instances_path}\nFiles loaded:')
for file in files:
    print(file)

In [ ]:
%autoreload

seed = 0 # 1
seed_stochastic_modules_globally(default_seed=seed)

env = EcoleBranching(observation_function='default',
                      information_function='default',
                      reward_function='default',
                      scip_params='default')
env.seed(seed)

ecole.seed(seed)

stats_to_record = ['Episode Length', 'Episode Return']
plot_dict = {
             'Original': {stat: []  for stat in stats_to_record},
             'Retrospective': {stat: []  for stat in stats_to_record},
            }

In [ ]:
%autoreload

subtree_reward_key = 'retro_binary_fathomed'
for i, file in enumerate(files):
    start = time.time()
    
    env.seed(seed)
    instance = ecole.scip.Model.from_file(file)
    agent.before_reset(instance)
    obs, action_set, reward, done, info = env.reset(instance)
    print(f'\nStarting episode {i+1} of {len(files)}...')
    
    step_counter = 0
    while not done:
        action, action_idx = agent.action_select(action_set, model=env.model, done=done)
        obs, action_set, reward, done, info = env.step(action)
        step_counter += 1
        
    print(f'Completed episode {i+1} in {time.time() - start:.3f} s.')
        
    # update whole tree stats
    plot_dict['Whole tree']['Episode Length'].append(step_counter)
    whole_tree_return = 0
    for sub_ep in reward[subtree_reward_key]:
        for r in sub_ep.values():
            whole_tree_return += r
    plot_dict['Whole tree']['Episode Return'].append(whole_tree_return)
    
    # update sub-tree stats
    for sub_ep in reward[subtree_reward_key]:
        plot_dict['Sub-trees']['Episode Length'].append(len(list(sub_ep.keys())))
        plot_dict['Sub-trees']['Episode Return'].append(sum(list(sub_ep.values())))
        
print(f'\nFinal plot_dict:\n{plot_dict}')

In [ ]:
# save plot dict
with open('subtree_vs_whole_tree_episode_length_plot_dict.pkl', 'wb') as f:
    pickle.dump(plot_dict, f)

In [ ]:
# load plot dict
with open('subtree_vs_whole_tree_episode_length_plot_dict.pkl', 'rb') as f:
    plot_dict = pickle.load(f)

In [ ]:
# rename keys if needed
if 'Whole tree' in plot_dict.keys():
    plot_dict['Retrospective'] = plot_dict.pop('Whole tree')
if 'Sub-trees' in plot_dict.keys():
    plot_dict['Original'] = plot_dict.pop('Sub-trees')

In [ ]:
# Create dataframes (from above dict) for data manipulation and seaborn plotting (use display_name for legends)
_df = defaultdict(list)
for _agent in plot_dict.keys():
    for stat in plot_dict[_agent].keys():
        try:
            for el in plot_dict[_agent][stat]:
                _df[stat].append(el)
        except TypeError:
            # non-iterable, is already an element
            _df[stat].append(plot_dict[_agent][stat])
            
    for _ in range(len(plot_dict[_agent][stat])):
        _df['Agent'].append(_agent)
                        
for key, val in _df.items():
    print(key, len(val))
df = pd.DataFrame(_df)
if verbose:
    print(tabulate(df, headers='keys', tablefmt='psql', showindex=False))

In [ ]:
# # config
# context = 'paper'
# style = 'whitegrid'
# font_scale = 2
# palette = 'hls'
# fig_size = (4, 2)
# linewidth = 2.5

# sns.set_theme(font_scale=font_scale,
#               context=context,
#               style=style,
#               palette=palette)


# plot_legend = False

# ylog = True
# xlog = True

In [ ]:
# config
ylabel = 'Proportion'

plot_legend = True

ylog = True
xlog = True

scaling_factor = 0.6
width_scaling_factor = 1.5
height_scaling_factor = 1

aesthetics = PlotAesthetics()
aesthetics.set_icml_paper_plot_aesthetics()

In [ ]:
# sns.set_theme(font_scale=font_scale,
#               context=context,
#               style=style,
#               palette=palette)


# EPISODE LENGTH
f, ax = plt.subplots(figsize=aesthetics.get_standard_fig_size(scaling_factor=scaling_factor, width_scaling_factor=width_scaling_factor, height_scaling_factor=height_scaling_factor))
g = sns.histplot(data=df, x='Episode Length', hue='Agent', log_scale=xlog, legend=plot_legend)
plt.xlabel('MDP Length')
plt.ylabel(ylabel)
ax.tick_params(axis='both', which='major', pad=2)
ax.xaxis.labelpad = 2
ax.yaxis.labelpad = 2
sns.despine(ax=ax) # remove top and right spines
if ylog:
    g.set(yscale='log')
if plot_legend:
    plt.legend(title=None, labels=list(plot_dict.keys()), loc='lower right')
plt.gcf().patch.set_alpha(0.0)
plt.show()

f, ax = plt.subplots(figsize=aesthetics.get_standard_fig_size(scaling_factor=scaling_factor, width_scaling_factor=width_scaling_factor, height_scaling_factor=height_scaling_factor))
g = sns.histplot(data=df, x='Episode Length', hue='Agent', log_scale=xlog, cumulative=True, stat='density', fill=False, common_norm=False, element='step', legend=plot_legend, linewidth=aesthetics.linewidth)
plt.xlabel('MDP Length')
plt.ylabel(ylabel)
ax.tick_params(axis='both', which='major', pad=2)
ax.xaxis.labelpad = 2
ax.yaxis.labelpad = 2
sns.despine(ax=ax) # remove top and right spines
if plot_legend:
    plt.legend(title=None, labels=list(plot_dict.keys()))
plt.gcf().patch.set_alpha(0.0)
plt.show()


# EPISODE RETURN
f, ax = plt.subplots(figsize=aesthetics.get_standard_fig_size(scaling_factor=scaling_factor, width_scaling_factor=width_scaling_factor, height_scaling_factor=height_scaling_factor))
g = sns.histplot(data=df, x='Episode Return', hue='Agent', legend=plot_legend)
plt.xlabel('MDP Return')
plt.ylabel(ylabel)
if ylog:
    g.set(yscale='log')
plt.xscale('symlog')
plt.xlim(right=0)
ax.tick_params(axis='both', which='major', pad=2)
ax.xaxis.labelpad = 2
ax.yaxis.labelpad = 2
sns.despine(ax=ax) # remove top and right spines
if plot_legend:
    plt.legend(title=None, labels=list(plot_dict.keys()))
plt.gcf().patch.set_alpha(0.0)
plt.show()

f, ax = plt.subplots(figsize=aesthetics.get_standard_fig_size(scaling_factor=scaling_factor, width_scaling_factor=width_scaling_factor, height_scaling_factor=height_scaling_factor))
g = sns.histplot(data=df, x='Episode Return', hue='Agent', cumulative=True, stat='density', fill=False, common_norm=False, element='step', legend=plot_legend, linewidth=aesthetics.linewidth)
plt.xlabel('MDP Return')
plt.ylabel(ylabel)
plt.xscale('symlog')
plt.xlim(right=0)
ax.tick_params(axis='both', which='major', pad=2)
ax.xaxis.labelpad = 2
ax.yaxis.labelpad = 2
sns.despine(ax=ax) # remove top and right spines
if plot_legend:
    plt.legend(title=None, labels=list(plot_dict.keys()), loc='upper left')
plt.gcf().patch.set_alpha(0.0)
plt.show()

In [ ]:
for agent in plot_dict.keys():
    for stat in plot_dict[agent].keys():
        print(f'{agent} -> mean {stat}: {np.mean(plot_dict[agent][stat])} | std {stat}: {np.std(plot_dict[agent][stat])}')